# Pipelines

`pipeline` 是使用模型进行推理的一种非常简单的方法。这些 `pipeline` 是从库中抽象出大部分复杂代码的对象，提供专用于多项任务的简单 API，包括命名实体识别、掩码语言建模、情感分析、特征提取和问答。有关使用示例，请参阅任务摘要。 

有两类管道抽象需要注意： 

- pipeline() 是封装所有其他管道的最强大的对象。
- 特定于任务的管道可用于音频、计算机视觉、自然语言处理和多模式任务。

In [ ]:
# !pip install transformers datasets torch

In [5]:
from transformers import pipeline

## 简单应用

只指定task的identifier, 甚至 `task` 这个参数名都可以省略。常见的 task identifier 详见[这里](https://huggingface.co/docs/transformers/main_classes/pipelines)。

![task](img/task.png)

In [ ]:
classifier = pipeline(task="sentiment-analysis") 
# classifier = pipeline("sentiment-analysis") 可以缩写成这样最简单的方式。

In [6]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

也可以指定模型，可以不写task的identifier。

In [8]:
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

[[{'label': 'sadness', 'score': 0.0005138218402862549}, {'label': 'joy', 'score': 0.9972521662712097}, {'label': 'love', 'score': 0.0007443306967616081}, {'label': 'anger', 'score': 0.0007404911448247731}, {'label': 'fear', 'score': 0.0003293847548775375}, {'label': 'surprise', 'score': 0.0004197480739094317}]]


/opt/homebrew/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


如果要判断中文语句的情感分析，还是要先使用翻译模型把语句翻译成英文。 这里选择 `Helsinki-NLP/opus-mt-zh-en` 这个模型来进行中译英

In [14]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-zh-en", max_time=7)

In [15]:
# 测试一下效果
prediction = translator("经过简单的站内搜索，我找到了下面三个模型，为了验证模型效果，我写了一句比较无厘头的测试内容，对这些模型进行测试。", )[0]["translation_text"]
print(prediction)

After a simple inside-of-site search, I found the following three models and, in order to verify their effects, I wrote a more unwieldy test to test them.


### 中文语句情感测试

In [18]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-zh-en")
eng_result = translator("扎心了，老铁", )[0]["translation_text"]
print(eng_result)

That's it, Iron.


In [19]:
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion')
prediction = classifier(eng_result)
print(prediction)

[{'label': 'anger', 'score': 0.47998687624931335}]


## handle French text

In [1]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': '5 stars', 'score': 0.7272654175758362}]